In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
from mlxtend.frequent_patterns import apriori, association_rules
import sys



# Pre processing

In [2]:
file_path = '/Users/leonardosantana/Documents/Power bi templates/Online Retail.xlsx'
df = pd.read_excel(file_path)

In [3]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

df = df[df['Quantity'] > 0]  
df = df[df['UnitPrice'] > 0]

In [4]:
# Convert InvoiceDate to a datetime object
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Extract day, year, time and hour
df['Day'] = df['InvoiceDate'].dt.day
df['Month'] = df['InvoiceDate'].dt.month
df['Year'] = df['InvoiceDate'].dt.year
df['Time'] = df['InvoiceDate'].dt.time
df['Hour'] = df['InvoiceDate'].dt.hour

In [5]:
# create a new column called "TotalPrice" by multiplying Quantity and Price
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [6]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Day,Month,Year,Time,Hour,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,1,12,2010,08:26:00,8,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1,12,2010,08:26:00,8,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,1,12,2010,08:26:00,8,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1,12,2010,08:26:00,8,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,1,12,2010,08:26:00,8,20.34


In [7]:
output_path = '/Users/leonardosantana/Directory/Project1/Online_retail_updated.csv'
df.to_csv(output_path, index=False)

# RFM - Costumer Segmentation

In [86]:
today_date = dt.datetime(2011,12,11)

In [87]:
rfm = df.groupby('CustomerID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                                'InvoiceNo': lambda num: num.nunique(),
                                                "TotalPrice": lambda price: price.sum()})

rfm.head()

,InvoiceDate,InvoiceNo,TotalPrice
CustomerID,,,
12346.0,326,1,77183.60
12347.0,3,7,4310.00
12348.0,76,4,1797.24
12349.0,19,1,1757.55
12350.0,311,1,334.40


In [88]:
rfm.columns = ['recency', 'frequency', "monetary"]
rfm.head()

,recency,frequency,monetary
CustomerID,,,
12346.0,326,1,77183.60
12347.0,3,7,4310.00
12348.0,76,4,1797.24
12349.0,19,1,1757.55
12350.0,311,1,334.40


In [89]:
rfm = rfm[(rfm['monetary'] > 0)]

In [90]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
CustomerID,,,,,,
12346.0,326,1,77183.60,1,1,5
12347.0,3,7,4310.00,5,5,5
12348.0,76,4,1797.24,2,4,4
12349.0,19,1,1757.55,4,1,4
12350.0,311,1,334.40,1,1,2


In [91]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str)+ rfm['monetary_score'].astype(str))

In [92]:
seg_map = {
    # High-Value Customers
    r'^55[345]$': 'VIP Customers',

    # Loyal Customers
    r'^[234]55$': 'Loyal Customers',
    r'^[4-5][3-4][4-5]$': 'Loyal Customers',
    r'^443$': 'Loyal Customers',
    r'^54[123]$': 'Loyal Customers',
    r'^45[3-4]$': 'Loyal Customers',
    r'^55[1-2]$': 'Loyal Customers',
    r'^452$': 'Loyal Customers',

    # Potential Loyalists
    r'^[234][3-4][4-5]$': 'Promising Customers',
    r'^1[3-4][3-4]$': 'Promising Customers',
    r'^[234]33$': 'Promising Customers',
    r'^4[3-4]2$': 'Promising Customers',
    r'^[123]54$': 'Promising Customers',
    r'^54[2-3]$': 'Promising Customers',
    r'^[2-3]43$': 'Promising Customers',  
    
    r'^251$': 'Promising Customers',  
    r'^[2-3]53$': 'Promising Customers',
    r'^34[1-2]$': 'Promising Customers',
    r'^155$': 'Promising Customers',

    # At-Risk Customers
    r'^[1-2][3-4][1-2]$': 'At-Risk Customers',
    r'^[345]32$': 'At-Risk Customers',
    r'^533$': 'At-Risk Customers',
    r'^[3-4]31$': 'At-Risk Customers', 
    r'^152$': 'At-Risk Customers',
    r'^145$': 'At-Risk Customers',
    r'^153$': 'At-Risk Customers',

    # New Customers
    r'^5[1-2][1234]$': 'New Customers',
    r'^[34][1-2][123]$': 'promising_customers',
    r'^3[1-2][12345]$': 'about_to_disengage',
    r'^4[1-2][12345]$': 'about_to_disengage',  
    r'^135$': 'about_to_disengage',
    r'^531$': 'about_to_disengage',
    r'^5[1-2]5$': 'about_to_disengage',

    # Inactive Customers
    r'^[1-2][1-2][4-5]$': 'Inactive Customers',

    # Churned Customers
    r'^[1-2][1-2][123]$': 'Churned Customers',
}



In [93]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
CustomerID,,,,,,,,
12346.0,326,1,77183.60,1,1,5,115,Inactive Customers
12347.0,3,7,4310.00,5,5,5,555,VIP Customers
12348.0,76,4,1797.24,2,4,4,244,Promising Customers
12349.0,19,1,1757.55,4,1,4,414,about_to_disengage
12350.0,311,1,334.40,1,1,2,112,Churned Customers


In [94]:
rfm1 = rfm[["segment","recency","frequency","monetary"]].groupby("segment").agg(["mean","count"])
rfm1

recency        frequency           monetary      
                           mean count       mean count         mean count
segment                                                                  
At-Risk Customers    111.704420   362   2.364641   362   379.273729   362
Churned Customers    220.433468   992   1.082661   992   300.780334   992
Inactive Customers   182.088608    79   1.341772    79  2833.879380    79
Loyal Customers       23.899083   872   6.288991   872  3101.278900   872
New Customers          7.670103    97   1.319588    97   436.701753    97
Promising Customers   89.042999   907   3.447630   907  1202.311357   907
VIP Customers          6.189931   437  16.086957   437  9227.497574   437
about_to_disengage    46.289157    83   1.783133    83  2333.007470    83
promising_customers   41.860511   509   1.137525   509   323.052083   509

In [107]:
rfm['CustomerID'] = rfm.index

rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment,CustomerID
CustomerID,,,,,,,,,
12346.0,326,1,77183.60,1,1,5,115,Inactive Customers,12346.0
12347.0,3,7,4310.00,5,5,5,555,VIP Customers,12347.0
12348.0,76,4,1797.24,2,4,4,244,Promising Customers,12348.0
12349.0,19,1,1757.55,4,1,4,414,about_to_disengage,12349.0
12350.0,311,1,334.40,1,1,2,112,Churned Customers,12350.0


In [108]:
rfm

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment,CustomerID
CustomerID,,,,,,,,,
12346.0,326,1,77183.60,1,1,5,115,Inactive Customers,12346.0
12347.0,3,7,4310.00,5,5,5,555,VIP Customers,12347.0
12348.0,76,4,1797.24,2,4,4,244,Promising Customers,12348.0
12349.0,19,1,1757.55,4,1,4,414,about_to_disengage,12349.0
12350.0,311,1,334.40,1,1,2,112,Churned Customers,12350.0
...,...,...,...,...,...,...,...,...,...
18280.0,278,1,180.60,1,2,1,121,Churned Customers,18280.0
18281.0,181,1,80.82,1,2,1,121,Churned Customers,18281.0
18282.0,8,2,178.05,5,3,1,531,about_to_disengage,18282.0


In [109]:
output_path = '/Users/leonardosantana/Directory/Project1/rfm_data.csv'
rfm.to_csv(output_path, index=False)
